#### 自动化生成latex主表中的关键数值部分

In [ ]:
content = r"""
\hline
\fairColor & Manual& 20.0 & 24.0\sred{+4.0} & & 22.6 & 25.9\sred{+3.3} & & 20.1 & 24.2\sred{+4.1}\\
% \cline{2-6}
\fairColor ~ & LPAQA & 19.7 & 23.7\sred{+4.0} & & 21.6 & 23.7\sred{+2.1} & & 20.1 & 23.5\sred{+3.4}\\
% \cline{2-6}
\fairColor ~ & AutoPrompt & 20.6 & 23.7\sred{+3.1} & & 21.2 & 24.2\sred{+3.0} & & 18.8 & 23.5\sred{+4.7}\\
% \cline{2-6}
\fairColor \multirow{-4}*{WIKI-UNI}  & OptiPrompt & 23.1 & 25.0\sred{+1.9} & &  24.9 &  27.8\sred{+2.9} & & 21.9 &  26.0\sred{+4.1}\\

\hline
\biasedColor  & Manual & 37.1 & 30.0\sblue{-7.1} & & 38.7 & 30.7\sblue{-8.0} & & 36.4 & 31.0\sblue{-5.4} \\
% \cline{2-6}
\biasedColor ~ & LPAQA & 38.1 & 28.2\sblue{-9.9} & & 40.2 & 29.1\sblue{-11.1} & & 39.0 & 30.6\sblue{-8.4}\\
% \cline{2-6}
\biasedColor ~ & AutoPrompt & 43.9 & 27.2\sblue{-16.7} & & 43.5 & 31.1\sblue{-12.4} & & 42.3 & 26.1\sblue{-16.2}\\
% \cline{2-6}
\biasedColor \multirow{-4}*{LAMA} & OptiPrompt & 49.4 & 30.9\sblue{-18.5} & & 52.6 &  33.5\sblue{-19.1} & & 48.0 &  34.2\sblue{-13.8}\\


\hline
\biasedColor  & Manual & 27.3 & 19.9\sblue{-7.4} & & 30.0 & 21.8\sblue{-8.2} & & 28.3 & 22.2\sblue{-6.1}\\
% \cline{2-6}
\biasedColor ~ & LPAQA & 29.0 & 18.5\sblue{-10.5} & & 31.7 & 20.4\sblue{-11.3} & & 31.1 & 21.9\sblue{-9.2}\\
% \cline{2-6}
\biasedColor ~ & AutoPrompt & 33.4 & 16.2\sblue{-17.2} & & 33.8 & 21.4\sblue{-12.4} & & 33.9 & 17.7\sblue{-16.2}\\
% \cline{2-6}
\biasedColor \multirow{-4}*{LAMA-UHN} & OptiPrompt & 39.5 & 19.8\sblue{-19.7} & &  43.8 &  23.2\sblue{-20.6} & & 39.3 &  24.4\sblue{-14.9} \\
"""

In [20]:
# 导入debias数据
import json
prompts = ["LAMA","LPAQA","AutoPrompt","optiprompt"]
models = ["bert-base-cased","bert-large-cased","roberta-large"]

no_norm_no_rescale = {}
for model in models:
    no_norm_no_rescale[model] = {}
    for prompt in prompts:
        vocab = "common_vocab_cased" if model != "roberta-large" else "common_vocab_cased_be_ro_al"
        results_path = f"/mnt/code/users/xuziyang/PromptBias/results/filter_out_0_biased_tokens/{model}/{vocab}/typed_querying/ablations/{prompt}/no_normal_no_rescale_result.json"
        # 解析result
        with open(results_path) as f:
            results = json.load(f)
        no_norm_no_rescale[model][prompt] = results

In [21]:
content = ""
for benchmark in ["WIKI-UNI", "LAMA", "LAMA-UHN"]:
    content += r"\hline"
    content += "\n"
    for prompt in ["Manual", "LPAQA", "AutoPrompt", "OptiPrompt"]:
        background = "biasedColor" if benchmark != "WIKI-UNI" else "fairColor"
        if prompt=="OptiPrompt":
            content += r"\{2} \multirow{{-4}}*{{ {0} }} & {1} &".format(benchmark,prompt,background)
        else:
            content += r"\{1} & {0} &".format(prompt,background)
        for model in ["bert-base-cased", "bert-large-cased","roberta-large"]:
            vanilla_acc, debiased_acc, diff = 0,0,0
            if prompt=="Manual":
                r_prompt = "LAMA"
            elif prompt=="OptiPrompt":
                r_prompt = "optiprompt"
            else:
                r_prompt = prompt
            r_benchmark = "LAMA-WHU" if benchmark=="LAMA-UHN" else benchmark
            vanilla_acc = next(iter(no_norm_no_rescale[model][r_prompt][model][r_benchmark].values()))["P"] * 100
            debiased_acc = next(iter(no_norm_no_rescale[model][r_prompt][model][r_benchmark].values()))["P_d"] * 100
            vanilla_acc = round(vanilla_acc,1)
            debiased_acc = round(debiased_acc,1)
            diff = debiased_acc-vanilla_acc
            diff  = round(diff, 1)
            diff  = "+"+str(diff) if diff>0 else str(diff)
            # vanilla_acc = round(vanilla_acc*10+0.5,)
            color = r"\sred" if benchmark=="WIKI-UNI" else r"\sblue"
            if model!="roberta-large":
                content += r" {0} & {1}{2}{{ {3} }} & &".format(vanilla_acc, debiased_acc, color, diff)
            else:
                content += r" {0} & {1}{2}{{ {3} }}\\".format(vanilla_acc, debiased_acc, color, diff)
        content+="\n"
            # "{prompt} &  22.6 & 25.9\sred{+3.3} & & 20.1 & 24.2\sred{+4.1}\\"

print(content)

\hline
\fairColor & Manual & 20.0 & 24.2\sred{ +4.2 } & & 22.6 & 26.1\sred{ +3.5 } & & 20.1 & 24.0\sred{ +3.9 }\\
\fairColor & LPAQA & 19.7 & 24.3\sred{ +4.6 } & & 21.6 & 24.4\sred{ +2.8 } & & 20.1 & 23.6\sred{ +3.5 }\\
\fairColor & AutoPrompt & 20.6 & 24.7\sred{ +4.1 } & & 21.2 & 25.0\sred{ +3.8 } & & 18.8 & 23.4\sred{ +4.6 }\\
\fairColor \multirow{-4}*{ WIKI-UNI } & OptiPrompt & 23.1 & 25.7\sred{ +2.6 } & & 24.8 & 28.3\sred{ +3.5 } & & 22.2 & 26.4\sred{ +4.2 }\\
\hline
\biasedColor & Manual & 37.1 & 32.4\sblue{ -4.7 } & & 38.7 & 32.2\sblue{ -6.5 } & & 36.4 & 30.7\sblue{ -5.7 }\\
\biasedColor & LPAQA & 38.1 & 31.0\sblue{ -7.1 } & & 40.2 & 31.6\sblue{ -8.6 } & & 39.0 & 30.5\sblue{ -8.5 }\\
\biasedColor & AutoPrompt & 43.9 & 33.4\sblue{ -10.5 } & & 43.5 & 34.8\sblue{ -8.7 } & & 42.3 & 26.1\sblue{ -16.2 }\\
\biasedColor \multirow{-4}*{ LAMA } & OptiPrompt & 49.7 & 34.1\sblue{ -15.6 } & & 52.4 & 38.6\sblue{ -13.8 } & & 48.5 & 34.7\sblue{ -13.8 }\\
\hline
\biasedColor & Manual & 27.2 & 22.

In [7]:
print(r"{{ {0} {1} }}".format("hello","hi"))

{ hello hi }
